In [180]:
from tensorflow import keras
from keras.datasets import mnist
import cv2
import numpy as np
import sys
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [181]:
(ax, ay), (qx, qy) = mnist.load_data()

In [182]:
axn = np.float32(np.empty(ax.shape))
axs = np.float32(np.empty(ax.shape))
axl = np.float32(np.empty(ax.shape))
axo = np.float32(np.empty(ax.shape))
qxn = np.float32(np.empty(qx.shape))
qxs = np.float32(np.empty(qx.shape))
qxl = np.float32(np.empty(qx.shape))
qxo = np.float32(np.empty(qx.shape))

In [183]:
length = ax.shape[0]
for i in range(length):
    axn[i, :-1, :] = ax[i, 1:, :]    # Deslocamento para norte
    axs[i, 1:, :] = ax[i, :-1, :]    # Deslocamento para sul
    axl[i, :, 1:] = ax[i, :, :-1]    # Deslocamento para leste
    axo[i, :, :-1] = ax[i, :, 1:]    # Deslocamento para oeste
    
new_ax = np.empty((5*length,28,28))
new_ax[:length,:,:] = ax[:,:,:]
new_ax[length:2*length,:,:] = axn[:,:,:]
new_ax[2*length:3*length,:,:] = axs[:,:,:]
new_ax[3*length:4*length,:,:] = axl[:,:,:]
new_ax[4*length:5*length,:,:] = axo[:,:,:]

In [185]:
length = qx.shape[0]
for i in range(length):
    qxn[i, :-1, :] = qx[i, 1:, :]    # Deslocamento para norte
    qxs[i, 1:, :] = qx[i, :-1, :]    # Deslocamento para sul
    qxl[i, :, 1:] = qx[i, :, :-1]    # Deslocamento para leste
    qxo[i, :, :-1] = qx[i, :, 1:]    # Deslocamento para oeste
    
new_qx = np.empty((5*length,28,28))
new_qx[:length,:,:] = qx[:,:,:]
new_qx[length:2*length,:,:] = qxn[:,:,:]
new_qx[2*length:3*length,:,:] = qxs[:,:,:]
new_qx[3*length:4*length,:,:] = qxl[:,:,:]
new_qx[4*length:5*length,:,:] = qxo[:,:,:]

In [187]:
new_ay = np.empty(5*ay.shape[0])
new_ay[:ay.shape[0]] = ay[:]
new_ay[ay.shape[0]:2*ay.shape[0]] = ay[:]
new_ay[2*ay.shape[0]:3*ay.shape[0]] = ay[:]
new_ay[3*ay.shape[0]:4*ay.shape[0]] = ay[:]
new_ay[4*ay.shape[0]:5*ay.shape[0]] = ay[:]

new_qy = np.empty(5*qy.shape[0])
new_qy[:qy.shape[0]] = qy[:]
new_qy[qy.shape[0]:2*qy.shape[0]] = qy[:]
new_qy[2*qy.shape[0]:3*qy.shape[0]] = qy[:]
new_qy[3*qy.shape[0]:4*qy.shape[0]] = qy[:]
new_qy[4*qy.shape[0]:5*qy.shape[0]] = qy[:]

In [189]:
new_ax = new_ax.astype('float32')
new_qx = new_qx.astype('float32')
new_ax = new_ax.reshape(new_ax.shape[0],new_ax.shape[1]*new_ax.shape[2])
new_qx = new_qx.reshape(new_qx.shape[0],new_qx.shape[1]*new_qx.shape[2])
new_ax /= 255.0 #0 a 1
new_qx /= 255.0 #0 a 1
qp=np.empty(new_qy.shape,dtype="uint8")

In [190]:
t1 = time.time()
FLANN_INDEX_KDTREE = 1
flann_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 4)
flann = cv2.flann_Index(new_ax, flann_params)
t2 = time.time()
matches, dists = flann.knnSearch(new_qx, 1)
t3 = time.time()

In [191]:
for l in range(matches.shape[0]):
    i=matches[l]
    qp[l]=new_ay[i]

In [192]:
erros=0;
for l in range(matches.shape[0]):
    if qp[l]!=new_qy[l]: erros+=1

In [193]:
print("Erros=%5.2f%%" % (100.0*erros/new_qy.shape[0]) )
print("Tempo de treinamento: %f"%(t2-t1))
print("Tempo de predicao: %f"%(t3-t2))

Erros= 4.11%
Tempo de treinamento: 15.989111
Tempo de predicao: 1.949744
